### Name: Mohan Sai Aravind Kumar
### Task 2

### Methodology:

1. Firstly through PyPDF2 package 30001781.pdf file is extracted. Unit codes are stored in a list, synopsis are stored in a list and the outcomes are also stored in list
2. For Case Normalization to takes place, I first considered doing sentence tokenization of each synopsis and outcomes list of strings using sent_tokenize and then normalized the word according to the asked question in the task
3. After that word tokenization is done on the lists before removing the STOPWORDS.
4. As the stopwords removed from the synopsis list and outcomes list, then both are concatenated which means each unitcode's synopsis and outcomes will be in one list stored another list.
5. Contexual words are removed but these word tokens are not repeated when compared with threshold frequency i.e 95%, so the words tokens are not removed
6. Removal of rare tokens are also done by chain.from_iterable() method and removed just like the stopwords removal from list
7. Unigrams and bi grams are generated using 'MWETokenizer'.
8. Stemming is done on the final tokens obtained after removal of rare tokens using PorterStemmer.
9. Finally generating vocab.txt and countvec.txt


### Importing all the Packages

In [266]:
import PyPDF2 # importing PyPDF2 python package to read and extract infromation in the pdf file 
import re     # importing re package in python to extract strings using regular expression

In [267]:
import nltk   # natural language toolkit
from nltk.tokenize import RegexpTokenizer # it tokenizes the given tokenizer(reg)
from nltk.tokenize import sent_tokenize  # it tokenizes each sentence  
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer # remove morphological affixes from words and leaves the stem
from nltk.probability import *
from __future__ import division
from itertools import chain  #chain.from_iterable() method is used to get the list of unique tokens
from nltk.util import ngrams # to create unigrams, bigrams, trigrams etc.

In [268]:
new_file = open('30001781.pdf','rb')    # opening given 30001781.pdf file
new_pdf = PyPDF2.PdfFileReader(new_file) # intializes PdfFileReader object
#print(new_pdf.getNumPages())    # to get how many pages are in the pdf(object.getNumPages)

In [269]:
unitcodes_list = []  #creating a unit codes list to store all the uit codes of pdf file in a list
for i in range(new_pdf.getNumPages()):  #new_pdf.getNumPages gives total number of pages i the pdf file 
    page_object = new_pdf.getPage(i) # storing each page of information in a page_object
    text=page_object.extractText() # page_object.extractText() method extract the infromation in page_object
    text=text.split('\n')
    for each in text:
        unit_code = re.findall(r'^[A-Z]{3}[0-9]{4}$',each) #regular expression for unitcode to extract from each pdf page
        if unit_code:    # findall gives all the matched patterns into a list of unit codes
            for item in unit_code:  
                unitcodes_list.append(item)  #all the matched patterns appended to list of all the unit codes
#len(unitcodes_list)


In [270]:
synopsis_list = []  # creating a list for synopsis to store each pdf unit codes 
for items in range(new_pdf.getNumPages()): # new_pdf.getNumPages gives total number of pages items the pdf file 
    page_object = new_pdf.getPage(items)  # storing each page of information in a page_object
    synopsis = re.compile(r'[A-Z]{3,4}[0-9]{4}(.*?)\[',re.DOTALL) #regular expression for synopsis to extract from each pdf page
    match_sys = re.findall(synopsis,page_object.extractText())  # for all the matached pattern , findall finds all those patterns and gives a list of synopsis
    if match_sys:
        for items in match_sys:    # each macthed synopsis is appended to synopsis_list
            synopsis_list.append(items)
#len(synopsis_list)
        

In [271]:
outcomes_list = []  # creating a list for outcomes to store each pdf unit codes 
for items in range(new_pdf.getNumPages()):  # new_pdf.getNumPages gives total number of pages items the pdf file 
    page_object = new_pdf.getPage(items)   # storing each page of information in a page_object
    outcomes = re.compile(r'[A-Z]{3}[0-9]{4}.*?(\[.*?\])',re.DOTALL)  #regular expression for outcomes to extract from each pdf page
    match_out = re.findall(outcomes,page_object.extractText())  # for all the matached pattern , findall finds all those patterns gives a list of outcomes
    if match_out:
        for item in match_out:   # each macthed outcomes is appended to outcomes_list
            outcomes_list.append(item)
            
#len(outcomes_list)

In [272]:
units_list = [] # final units_list created
for i in range(0,200):   # for each unit code, each synopsis and each outcome is appended to units_list 
    units_list.append([unitcodes_list[i],synopsis_list[i],outcomes_list[i]])

## Case Normalization

In [273]:
sentence_syn = []  # created senteneced synopsis list to append all the normalized token and ready for word tokenization.
for item in synopsis_list:
    text = sent_tokenize(item) # sentence tokenization is done here and forwarded for case normalization.
    sent_list=[]
    for each in text:
        each_list=each.split()
        each_list[0] = each_list[0].lower()  # case normalization lower is executed here
        joined = ' '.join(each_list)
        sent_list.append(joined)
    synopsis_sent = '.'.join(sent_list)
    sentence_syn.append(synopsis_sent)     

sentence_out = []  # created senteneced synopsis list to append all the normalized token and ready for word tokenization.
for items in outcomes_list:
    text1 = sent_tokenize(items)  # sentence tokenization is done here and forwarded for case normalization.
    sent_list1=[]
    for each1 in text:
        
        each_list1=each1.split()
        each_list1[0] = each_list1[0].lower()  # case normalization lower is executed here
        joined1 = ' '.join(each_list1)
        sent_list1.append(joined1)
    outcomes_sent = '.'.join(sent_list1)
    sentence_out.append(outcomes_sent) 
        

## Word Tokenization

In [276]:
synopsistokens_list = []  # creating synopsis tokens list for tokenized synopsis list
outcomestokens_list = []  # creating outcomes tokens list for tokenized outcomes list
tokenizer = RegexpTokenizer(r"\w+(?:[-']\w+)?")   # regular expression for word tokenization 
for value in sentence_syn:           
    tokens1 = tokenizer.tokenize(value)
    synopsistokens_list.append(tokens1)  # tokenized synopsis list
                                          
for item in outcomes_list:
    tokens = tokenizer.tokenize(item)
    outcomestokens_list.append(tokens)  # tokenized outcomes list
    


## Removing Stopwords

In [225]:
stopwords_list = []   # creating stopwords empty list
syns_stop=[]          # creating empty synopsis removed stopwords list
outs_stop=[]          # creating empty outcomes removed stopwords list
  # creating a empty list for combined synopsis and outcomes stopwords list
stopwords_list = open('stopwords_en.txt','r')
stopwords_set = set(stopwords_list)
stopwords_list1 = []
for each in stopwords_set:
    each=each.strip().replace('\n','')
    stopwords_list1.append(each)
       #converting list of stopwords to set of stopwords
for values2 in synopsistokens_list:       
    stop_sys_list=[]
    for each in values2: 
        if each not in stopwords_list1:     # removing stop words from tokenized synopsis words 
            stop_sys_list.append(each)
    syns_stop.append(stop_sys_list)       #appending to a list of stopwords removed synopsis tokens  
    
for values3 in outcomestokens_list:
    stop_out_list=[]
    for each in values3: 
        if each not in stopwords_list1:    # removing stop words from tokenized outcomes words 
            stop_out_list.append(each)
    outs_stop.append(stop_out_list)      #appending to a list of stopwords removed outcomes tokens 
              

Integrated both the list into tokens_sys_out i.e tokens of synopsis and outcomes in one list

In [277]:
tokens_sys_out = [] 
for i in range(len(syns_stop)):          # integrating both removed synopsis stop words and removed outcomes stop words
    tokens_sys_out.append(syns_stop[i]+outs_stop[i])


Most frequent words with respect to document frequency: that is count of how many number of documents a word occurs.
Instead iterating loops, we use FreqDist() and  set() as below

1. we use set() to get unique words in the tokens_sys_out
[set(value) for value in tokens_sys_out]
2. and we use chain.from_iterable to get list of words and then use FreqDist

In [278]:
words_count = list(chain.from_iterable([set(value) for value in tokens_sys_out]))
fd_2 = FreqDist(words_count)
#print("most common words:",fd_2.most_common(25))

As the above stats for document frequency shows that none of words match to threshold which is set to 95% i.e approximatley 199 words count.
None of the above match the count. so we don't remove any words from the tokens list 

here we are checking the term freqency that is count of words occuring in the entire tokes list 
1. we use chain.from_iterable to gets the words in tokens_sys_out but without using set()
2. use FreqDist() method to get the frequency ditribution of each words 

In [279]:
words_3 = list(chain.from_iterable(tokens_sys_out))  #list of words occuring the document 
fd_3 = FreqDist(words_3) # frequency ditribution of the words.

In [280]:
fd_3.most_common(10)

[('unit', 243),
 ('students', 178),
 ('research', 174),
 ('design', 172),
 ('skills', 146),
 ('management', 117),
 ('data', 105),
 ('analysis', 102),
 ('development', 100),
 ('understanding', 97)]

## Rare Tokens removal

As they asked for threshold set to 5%, that comes upto 10 
As we found the document frequecy of all the words in the document, we can now eailsy remove rare tokens whose document frequency is less than 10. It can be done as follows
1. Firstly, take the rare tokens out and append them to list so that we can remove them from the main list.
2. fd_3 is the dictionary of all the document frequencies, so we can remove from that dictionary, the rare tokens by comparing count of each key value to greater than 10 and apped them to list as shown below. 

In [282]:
less_freq_words = []
for each1 in fd_3:       # finding the less frequent words and appending to a list
    if fd_3[each1] < 10:
        less_freq_words.append(each1)

Finally, below cell contains code to remove the rare tokens from the tokens_sys_out

In [284]:
removed_sys_out = []         
for tokens in tokens_sys_out:               #removing the less frequent words from the list of all the removed stopped tokens
    less_sys_out=[]
    for each in tokens: 
        if each not in less_freq_words:
            less_sys_out.append(each)
    removed_sys_out.append(less_sys_out)

## uni-grams and bi-grams

here we are checking the term freqency that is count of words occuring in the entire tokes list using chain.from_iterable()



In [285]:
freq = list(chain.from_iterable(value for value in removed_sys_out))
bigrams = ngrams(freq, n = 2)  # importing ngrams method
fdbigram = FreqDist(bigrams)   # calculating frequency distribution 

In [286]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_words(freq)
bigrams_list = finder.nbest(bigram_measures.pmi, 200) # creating bigrams for frequency tokens

In [287]:
# A MWETokenizer merges multi_word expressions into single tokens
tokenizer = MWETokenizer(bigrams_list)   # so here variable takes MWEToeknizer with bigrams_list and converts into bigrams 
all_grams_list = []
for token in removed_sys_out:
    all_grams_list.append(tokenizer.tokenize(token))

In [288]:
stemmer = PorterStemmer()  # stemming the using PorterStemmer()
stemming_allgrams = []       
for items in all_grams_list:
    stem_list=[]
    for each in items:         #each item in the all_grams_list is stemmed 
        each=stemmer.stem(each)
        stem_list.append(each)
    stemming_allgrams.append(stem_list)

## Vocab.txt

1. Generating vocab.txt by taking stemming_allgrams list, applying chain.from_iterable() and make a set and sort it out in alphabatically order.
and the writing the vocab_file items into the .txt file
2. After stemming all the tokens , I converted into sorted list and then checking for the word length tokes less than 3 or equal to 3
3. wrtiting the value and index_value to the .txt file

In [289]:
vocab_file = sorted(set(list(chain.from_iterable(value for value in stemming_allgrams))))
for value in vocab_file:      # here removing the tokens with wordLength less than 3 or equal to 3
    if len(value) <= 3:       
        del vocab_file[vocab_file.index(value)]
with open('30001781_vocab.txt','w+') as file:    # opening the file here
    for value in vocab_file:
        file.write(f'{value}:{vocab_file.index(value)}\n') # writing the vocab_file items into the file

## CountVector

In this cell I created 30001781_countVec.txt to write all the unitcodes,vocab_file.index and stemming_allgrams.count()
So I have written all the document frequency values, indecies of particular unit.

In [290]:
with open('30001781_countVec.txt','w+') as vec:
    for p in range(len(stemming_allgrams)):
        vec.write(f'\n{unitcodes_list[p]},')
        for each in stemming_allgrams[p]:
            if each in vocab_file:
                vec.write(f'{vocab_file.index(each)}:{stemming_allgrams[p].count(each)},')